In [1]:
import rospy
import rosmsg
import rostopic
import rosservice
import os
import re
import configparser
import doctest

In [2]:
# Config

# Path to the generated server sources
OUT_DIR = os.path.abspath(os.path.join(os.path.abspath(''), 'generated'))
# TODO
SNAPSHOT_FILE = os.path.join(OUT_DIR, 'snapshot.ini')
# Path to the generated .proto file
PROTO_FILE = os.path.join(OUT_DIR, 'ros.proto')

In [3]:

config = configparser.ConfigParser()
# enable case-sensitive keys (for the ROS types)
config.optionxform=str
config['MESSAGE_DEFINITIONS'] = {}
config['TOPICS'] = {}
config['SERVICES'] = {}
config['ACTIONS'] = {}

published, subscribed = rostopic.get_topic_list()
published_topics = dict(map(lambda x: (x[0], x[1]), (published + subscribed))).items()
    
for (message_name, ros_type) in published_topics:
    config['TOPICS'][message_name] = ros_type
    config['MESSAGE_DEFINITIONS'][ros_type] = rosmsg.get_msg_text(ros_type)

services = rosservice.get_service_list()

for service_name in services:
    ros_type = rosservice.get_service_type(service_name)
    config['SERVICES'][service_name] = ros_type
    config['MESSAGE_DEFINITIONS'][ros_type] = rosmsg.get_srv_text(ros_type)
    
def flatten_types():
    changed = False
    
    for type_name in config['MESSAGE_DEFINITIONS'].keys():
        fields = config['MESSAGE_DEFINITIONS'][type_name].split('\n')
        top_level_fields = []
        sub_fields = []
        for field in fields:
            if field.startswith('  '):
                sub_fields.append(field)
            else:
                if sub_fields:
                    sub_type_name = top_level_fields[-1].split(' ')[0]
                    config['MESSAGE_DEFINITIONS'][sub_type_name] = '\n'.join(sub_fields)
                    sub_fields = []
                top_level_fields.append(field)

        config['MESSAGE_DEFINITIONS'][type_name] = '\n'.join(top_level_fields)
    
    if (changed):
        flatten_types()

flatten_types()
    
    
    
    
with open(SNAPSHOT_FILE, 'w') as configfile:
    config.write(configfile)
    


ROSServiceIOException: Unable to communicate with service [/turtlesim/get_loggers], address [rosrpc://azazdeaz-P65xHP:59277]

In [4]:
class RosSnapshot:
    def __init__(self):
        self.config = configparser.ConfigParser()
        # enable case-sensitive keys (for the ROS types)
        self.config.optionxform=str
        
    def load(self, path):
        self.config.read(path)
    
    def get_message_definitions(self):
        return self.config["MESSAGE_DEFINITIONS"]
    
    def get_message_definition_packages(self):
        ros_types = self.get_message_definitions().keys()
        return set(map(lambda t: t.split('/')[0], ros_types))
    
    def get_topics(self):
        """
        Returns Map<topic, ros_type>
        """
        return self.config["TOPICS"]
    
    def get_services(self):
        """
        Returns Map<service, ros_type>
        """
        return self.config["SERVICES"]

    def get_sections(self, ros_type):
        """
        Returns the parts of a ROS message as a list. (one part for topics, two for services, three for actions)
        """
        if ros_type not in self.config["MESSAGE_DEFINITIONS"]:
            raise KeyError("Can't find message definition for '{}'".format(ros_type))

        return self.config["MESSAGE_DEFINITIONS"][ros_type].split('---')
    
    def get_fields(self, ros_type, section=0):
        """
        Returns (ros_type, field_name)[]
        """
        sections = self.get_sections(ros_type)
        fields = sections[section].strip().split('\n')
        fields = filter(lambda f: f != '', fields)
        # skip constants
        fields = filter(lambda f: not '=' in f, fields)
        fields = map(lambda f: f.split(' '), fields)
        return fields

    def __str__(self):
        return '<RosSnapshot topics={} services={} message_definitions={}>'.format(
            len(self.get_topics()),
            len(self.get_services()),
            len(self.get_message_definitions()))
    

snap = RosSnapshot()
snap.load(SNAPSHOT_FILE)
print(list(snap.get_message_definitions()))
print(list(snap.get_fields('roscpp/GetLoggers')))

# print(snap.config["MESSAGE_DEFINITIONS"]["turtlesim/Spawn"].split('\n'), 1)
# list(snap.get_message_definition_packages())

ParsingError: Source contains parsing errors: '/home/azazdeaz/repos/grpc_bridge_test/src/grpc_api_generator/scripts/generated/snapshot.ini'
	[line 20]: '/baz srvs/Baz\n'

In [5]:
# Example ROS snapshot for unit tesing
TEST_SNAPSHOT_FILE = """
[MESSAGE_DEFINITIONS]
msgs/Foo = std_msgs/Header header
	string name
	uint32[] numbers
	msg/Bar[5] bar

msgs/Bar = uint8 number

srvs/Empty = ---

srvs/Baz = string logger
	---
	string level


[TOPICS]
/foo = msgs/Foo

[SERVICES]
/baz = srvs/Baz
"""
def tsnap():
    """
    Returns a RosSnapshot instance filled with the test data
    """
    snap = RosSnapshot()
    snap.config.read_string(TEST_SNAPSHOT_FILE)
    return snap

In [6]:
scalar_ros2pb = {
    'bool': 'bool',
    'int8': 'int32',
    'uint8': 'uint32',
    'int16': 'int32',
    'uint16': 'uint32',
    'int32': 'int32',
    'uint32': 'uint32',
    'int64': 'int64',
    'uint64': 'uint64',
    'float32': 'float',
    'float64': 'double',
    'string': 'string',
    'time': 'Time',
    'duration': 'Duration',
    'char': 'uint32',
    'byte': 'int32',
}

def write_file(path, content):
    folder = os.path.dirname(path)
    
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    f = open(os.path.join(path), 'w+')
    f.write(content)
    f.close()
    print("file was written to {}".format(path))

    # TODO remame strip_ros_array_notation
def strip_array_notation(name):
    """Remove array notation from ROS msg types
    
    >>> strip_array_notation('Foo[]')
    'Foo'
    >>> strip_array_notation('Foo[123]')
    'Foo'
    >>> strip_array_notation('Foo')
    'Foo'
    """
    return re.sub(r'\[\d*\]$', '', name)

def type_ros2pb(ros_type):
    """Convert ROS msg type to protobuff type
    
    >>> type_ros2pb('byte')
    'int32'
    >>> type_ros2pb('std_msgs/Header')
    'std_msgs.Header'
    >>> type_ros2pb('string[]')
    'repeated string'
    >>> type_ros2pb('time')
    'Time'
    >>> type_ros2pb('string')
    'string'
    >>> type_ros2pb('uint32')
    'uint32'
    """
    pb_type = strip_array_notation(ros_type)
    if pb_type in scalar_ros2pb:
        pb_type = scalar_ros2pb[pb_type]

    if ros_type.endswith(']'):
        pb_type = 'repeated ' + pb_type
    
    return pb_type.replace('/', '.')

def topic2service_name(topic):
    """Convert ROS topic names to valid protobuff names (replace slash with underscore)
    
    >>> topic2service_name('/rosout')
    'rosout'
    >>> topic2service_name('/rosout_agg')
    'rosout_agg'
    >>> topic2service_name('/turtle1/pose')
    'turtle1_pose'
    >>> topic2service_name('/turtle1/color_sensor')
    'turtle1_color_sensor'
    """
    return topic.replace('/', '_')[1:]

def parse_ros_type(ros_type):
    """Convert a line in a ROS msg file to (package, typename, is_array)
    
    >>> parse_ros_type('rosgraph_msgs/Log')
    ('rosgraph_msgs', 'Log', False)
    >>> parse_ros_type('turtlesim/Pose')
    ('turtlesim', 'Pose', False)
    >>> parse_ros_type('uint32')
    (None, 'uint32', False)
    >>> parse_ros_type('time')
    (None, 'time', False)
    >>> parse_ros_type('rosgraph_msgs/Log')
    ('rosgraph_msgs', 'Log', False)
    """
    is_array = ros_type.endswith(']')
    ros_type = strip_array_notation(ros_type)
    
    if '/' in ros_type:
        package, typename = ros_type.split('/')
        return package, typename, is_array
    else:
        return None, ros_type, is_array

import doctest
doctest.testmod()

TestResults(failed=0, attempted=18)

## Python script templates

In [7]:
topic_service_template = """
service {service_name} {{
    rpc Publish({pb_type}) returns (Empty);
    rpc Subscribe(Empty) returns (stream {pb_type});
}}

"""

srv_service_template = """
service {service_name} {{
    rpc Call({request_type}) returns ({response_type});
}}

"""

header = '''
syntax = 'proto3';

package ros;

message Empty {}

message Time {
    uint32 secs = 1;
    uint32 nsecs = 2;
}

message Duration {
    int32 secs = 1;
    int32 nsecs = 2;
}

'''

In [22]:
def generate_pb_message_definition_for_package(snap: RosSnapshot, pkg):
    """Gerate the proto definitions of a ros package
    >>> print(generate_pb_message_definition_for_package(tsnap(), "msgs"))
    message msgs {
      message Foo {
        std_msgs.Header header = 1;
        string name = 2;
        repeated uint32 numbers = 3;
        repeated msg.Bar bar = 4;
      }
      message Bar {
        uint32 number = 1;
      }
    }
    <BLANKLINE>
    <BLANKLINE>

    >>> print(generate_pb_message_definition_for_package(tsnap(), "srvs"))
    message srvs {
      message Empty {
        message req {
        }
        message res {
        }
      }
      message Baz {
        message req {
          string logger = 1;
        }
        message res {
          string level = 1;
        }
      }
    }
    <BLANKLINE>
    <BLANKLINE>
    """
    proto = 'message %s {\n' % pkg
    for ros_type in snap.get_message_definitions().keys():
        pkgname, msgname = ros_type.split('/')
        if pkg == pkgname:
            msgname = strip_array_notation(msgname)
            proto += '  message %s {\n' % msgname

            section_count = len(snap.get_sections(ros_type))
            is_msg = section_count == 1
            is_srv = section_count == 2
            for section in range(section_count):
                tabs = '    '
                if is_srv:
                    tabs = '      '
                    proto += '    message %s {\n' % ('req' if section == 0 else 'res')

                fields = snap.get_fields(ros_type, section)
                for key, (field_type, field_name) in enumerate(fields):
                    definition = '{} {} = {};'.format(
                        type_ros2pb(field_type), field_name, key+1)
                    proto += '{}{}\n'.format(tabs, definition)
                
                if is_srv:
                    proto += '    }\n'


            proto += '  }\n'
    proto += '}\n\n'
    return proto
doctest.testmod()
def generate_pb_message_all_definitions(snap: RosSnapshot):
    proto = ''
    for pkg in snap.get_message_definition_packages():
        proto += generate_pb_message_definition_for_package(snap, pkg)
    return proto
                

def generate_pb_service(topic, ros_type):
    return msg_service_template.format(service_name=topic2service_name(topic), pb_type=type_ros2pb(ros_type))

def generate_proto_file(snap: RosSnapshot):
    print('Found {} messages'.format(len(snap.get_message_definitions())))
    
    content = header
    
    content += generate_pb_message_all_definitions(snap)
            
        
    for (topic, ros_type) in snap.get_topics().items():
        content += topic_service_template.format(
            service_name=topic2service_name(topic), 
            pb_type=type_ros2pb(ros_type))

    # for (ros_type, topic) in snap.get_topics().items():
    #     content += srv_service_template.format(
    #         service_name=topic2service_name(topic), 
    #         request_type=type_ros2pb(ros_type),
    #         response_type=type_ros2pb(ros_type))
            
    write_file(os.path.join(PROTO_FILE), content)



    
generate_proto_file(snap)

Found 4 messages
file was written to /home/azazdeaz/repos/grpc_bridge_test/src/grpc_api_generator/scripts/generated/ros.proto


In [9]:
frame_template = '''
from concurrent import futures
import time
import math
import logging
import argparse
import sys
import threading
import time

import grpc

import rospy
import roslib.message

import ros_pb2 as ros_pb
import ros_pb2_grpc as ros_grpc


{classes}

def create_server():
    server = grpc.server(futures.ThreadPoolExecutor(max_workers=10))
{add_servicers}
    return server


def run_server(address):
    rospy.init_node('grpc_server', anonymous=True)
    server = create_server()
    server.add_insecure_port(address)
    server.start()
    print("gRPC server is running at %s" % address )
    rospy.spin()

def parse_args_and_run_server():
    run_server(parse_args())


def parse_args(args=None):
    parser = argparse.ArgumentParser(
        description='Generated ROS gRPC server')
    parser.add_argument(
        '-a', 
        '--address',
        help='host:port: host and port of the gRPC server to connect to', 
        default='[::]:50051')


    results = parser.parse_args(args)
    return results.address


if __name__ == '__main__':
    parse_args_and_run_server()

'''

add_servicer_template = 'ros_grpc.add_{servicer_class}_to_server({servicer_class}(), server)'

class_template = '''
class {servicer_class}(ros_grpc.{servicer_class}):
    def __init__(self):
        self.pub = None
        self.Msg = roslib.message.get_message_class('{ros_type}')

    def Publish(self, pb_msg, context):
        print('pb_msg', pb_msg, context)

        if self.pub == None:
            self.pub = rospy.Publisher('{topic}', self.Msg, queue_size=10)

        ros_msg = self.Msg()
{copy_pb2ros}
        print('publishing', ros_msg)
        self.pub.publish(ros_msg)
        return ros_pb.Empty()

    def Subscribe(self, request, context):
        c = {{'unsubscribed': False}}
        ros_messages = []

        def callback(ros_msg):
            print('ros in', ros_msg)
            ros_messages.append(ros_msg)
        subscription = rospy.Subscriber('{topic}', self.Msg, callback)

        def on_rpc_done():
            c['unsubscribed'] = True
            print("Attempting to regain servicer thread...", c)
            subscription.unregister()

        context.add_callback(on_rpc_done)

        print('sub', request, context)
        while not c['unsubscribed']:
            while ros_messages:
                ros_msg = ros_messages.pop(0)
{copy_ros2pb}
                yield pb_msg
            rospy.sleep(0.01)


'''

def add_tab(lines, tabs=1):
    return re.sub(r'^([^$])',  '    ' * tabs + '\\1', lines, flags=re.MULTILINE)


def generate_msg_copier(snap: RosSnapshot, ros_type, left='pb_msg', right='ros_msg', new_instance=True):
    result = ''
    package, typename, _ = parse_ros_type(ros_type)
    
    if new_instance:
        if package:
            if left.startswith('ros_'):
                result += '{} = roslib.message.get_message_class(\'{}/{}\')()\n'.format(
                    left, package, typename)
            else:
                result += '{} = ros_pb.{}.{}()\n'.format(left, package, typename)
        else:
            result += '{} = ros_pb.{}()\n'.format(left, typename)

    if snap.get_fields(ros_type): #TODO is this needed
        for ros_fieldtype, fieldname in snap.get_fields(ros_type):
            package, typename, is_array = parse_ros_type(ros_fieldtype)
            print('package, typename, is_array', package, typename, is_array)

            is_complex = package is not None # TODO is_scalar

            if is_array:
                sub_left = '{}_'.format(left.split('.')[0])
                sub_right = '{}_'.format(right.split('.')[0])
                result += 'for {sub_right} in {right}.{fieldname}:\n'.format(
                    sub_right=sub_right, right=right, fieldname=fieldname)
                body = ''
                if is_complex:
                    body += generate_msg_copier(
                        snap, ros_fieldtype, sub_left, sub_right, True)
                    body += '{left}.{fieldname}.append({sub_left})\n'.format(
                        left=left, sub_left=sub_left, fieldname=fieldname)
                else:
                    body += '{left}.{fieldname}.append({sub_right})\n'.format(
                        left=left, sub_right=sub_right, fieldname=fieldname)
                result += add_tab(body)
            elif is_complex:
                sub_left = '{}.{}'.format(left, fieldname)
                sub_right = '{}.{}'.format(right, fieldname)
                result += generate_msg_copier(snap, ros_fieldtype, sub_left, sub_right, False)
            else:
                result += '{left}.{fieldname} = {right}.{fieldname}\n'.format(
                    left=left, right=right, fieldname=fieldname)

    return result


def generate_server(snap):
    add_servicers = []
    servicer_classes = []
    for topic, ros_type in snap.get_topics().items():
        servicer_class = topic2service_name(topic) + 'Servicer'
        add_servicers.append(add_servicer_template.format(
            servicer_class=servicer_class))
        copy_ros2pb = generate_msg_copier(
            snap, ros_type, 'pb_msg', 'ros_msg', new_instance=True)
        copy_pb2ros = generate_msg_copier(
            snap, ros_type, 'ros_msg', 'pb_msg', new_instance=False)
        copy_ros2pb = add_tab(copy_ros2pb, 4)
        copy_pb2ros = add_tab(copy_pb2ros, 2)
        servicer_classes.append(class_template.format(
            servicer_class=servicer_class, ros_type=ros_type, topic=topic, copy_ros2pb=copy_ros2pb, copy_pb2ros=copy_pb2ros))

    return frame_template.format(add_servicers=add_tab('\n'.join(add_servicers)), classes='\n'.join(servicer_classes))

write_file(os.path.join(OUT_DIR, '__init__.py'), "")
write_file(os.path.join(OUT_DIR, 'grpc_server.py'), 
           generate_server(snap))
print('grpc_server.py file generated')


file was written to /home/azazdeaz/repos/grpc_bridge_test/src/grpc_api_generator/scripts/generated/__init__.py
package, typename, is_array std_msgs Header False
package, typename, is_array None uint32 False
package, typename, is_array None time False
package, typename, is_array None string False
package, typename, is_array None byte False
package, typename, is_array None string False
package, typename, is_array None stringFalse
package, typename, is_array None string False
package, typename, is_array None string False
package, typename, is_array None uint32 False
package, typename, is_array None string True
package, typename, is_array std_msgs Header False
package, typename, is_array None uint32 False
package, typename, is_array None time False
package, typename, is_array None string False
package, typename, is_array None byte False
package, typename, is_array None string False
package, typename, is_array None string False
package, typename, is_array None string False
package, typename

In [10]:
snap


In [11]:
!python3 -m grpc_tools.protoc \
    -I={os.path.relpath(OUT_DIR)} \
    --python_out={os.path.relpath(OUT_DIR)} \
    --grpc_python_out={os.path.relpath(OUT_DIR)} \
    {os.path.relpath(PROTO_FILE)}

ros.proto:38:17: Expected "{".


### All done! Run `rosrun grpc_api_generator run_server` to kick in the gRPC server.